# Part 1

Check for doubles.

In [1]:
io.Source.fromFile("input").getLines
    .map(line => line.split("bag", 2)(0)).toList
    .groupBy(identity).filter(_._2.size > 1)

res0: Map[String, List[String]] = Map()

In [2]:
val ruleRegex = raw"(\w+ \w+) bag".r

val containRules = io.Source.fromFile("input").getLines
    .map{line => {
        val bags = ruleRegex.findAllMatchIn(line).map(_.group(1)).toList
        bags.head -> bags.tail.toSet
    }}.toList

containRules.size

ruleRegex: scala.util.matching.Regex = (\w+ \w+) bag
containRules: List[(String, Set[String])] = List(
  ("light chartreuse", Set("mirrored yellow", "vibrant violet")),
  (
    "dotted silver",
    Set("dotted orange", "bright fuchsia", "bright tomato", "faded turquoise")
  ),
  ("plaid indigo", Set("pale violet", "mirrored violet")),
  ("faded turquoise", Set("faded lavender")),
  ("striped yellow", Set("vibrant black", "mirrored gold")),
  ("dull tan", Set("vibrant aqua")),
  ("striped brown", Set("vibrant orange", "posh bronze", "dim purple")),
  ("bright teal", Set("muted cyan", "drab white", "drab gray")),
  ("wavy tan", Set("wavy tomato", "dim tan")),
  (
    "light lavender",
    Set("dotted black", "wavy maroon", "pale white", "clear tan")
  ),
  (
    "dull orange",
    Set("bright brown", "muted gold", "vibrant black", "posh bronze")
  ),
  (
    "posh cyan",
    Set("mirrored tomato", "light salmon", "faded brown", "vibrant red")
  ),
  ("dark lavender", Set("faded purple"))

Invert bag-container relationship to walk graph from `shiny gold` upstream.

In [3]:
val directContainers = containRules
    .flatMap{case (container, bags) => bags.map(_ -> container)}
    .groupBy(_._1)
    .map{case (bag, bcPairs) => bag -> bcPairs.map(_._2).toSet}

directContainers("shiny gold")

directContainers: Map[String, Set[String]] = Map(
  "dull violet" -> Set(
    "dark yellow",
    "posh teal",
    "dim magenta",
    "dull purple",
    "posh magenta",
    "mirrored silver",
    "wavy aqua"
  ),
  "mirrored red" -> Set("muted silver", "plaid lavender"),
  "posh fuchsia" -> Set("muted indigo", "bright chartreuse", "shiny tan"),
  "dull salmon" -> Set("plaid blue", "dim turquoise", "plaid orange", "dark purple"),
  "vibrant beige" -> Set("dotted green", "vibrant cyan", "dim gold"),
  "shiny yellow" -> Set("vibrant silver", "faded yellow", "dotted brown"),
  "wavy gray" -> Set("plaid aqua"),
  "dull blue" -> Set(
    "striped crimson",
    "vibrant orange",
    "dull silver",
    "drab gold",
    "drab indigo"
  ),
  "striped indigo" -> Set(
    "dark indigo",
    "dotted purple",
    "pale cyan",
    "striped beige",
    "dotted fuchsia",
    "bright salmon",
    "shiny fuchsia",
    "faded fuchsia",
    "clear tan"
  ),
  "plaid bronze" -> Set(
    "dull yellow",
    "d

Walk starting from `shiny gold` upstream and collect containers we visit.

In [4]:
var front = Set("shiny gold")
var visited = Set[String]()

while (front.nonEmpty) {
    val candidates = front.flatMap(directContainers.getOrElse(_, Set()))
    visited ++= front
    front = candidates -- visited
}

(visited - "shiny gold").size

front: Set[String] = Set()
visited: Set[String] = Set(
  "dull violet",
  "mirrored red",
  "vibrant beige",
  "faded gray",
  "dull blue",
  "dull black",
  "striped black",
  "vibrant teal",
  "vibrant lavender",
  "bright lavender",
  "posh black",
  "pale magenta",
  "drab bronze",
  "clear aqua",
  "muted teal",
  "plaid orange",
  "mirrored coral",
  "dotted bronze",
  "pale red",
  "clear bronze",
  "plaid olive",
  "dull white",
  "bright blue",
  "dim coral",
  "dotted violet",
  "dark yellow",
  "faded beige",
  "light aqua",
  "posh purple",
  "muted tomato",
  "shiny maroon",
  "muted blue",
  "dull indigo",
  "posh teal",
  "plaid yellow",
  "wavy violet",
  "dull teal",
  "striped crimson",
...
res3_3: Int = 252

# Part 2

In [5]:
val ruleRegex = raw"(\w+ \w+) bags contain (.*)".r
val bagRegex = raw"(\d+) (\w+ \w+) bag".r


val containRules = io.Source.fromFile("input").getLines
    .map{
        case ruleRegex(color, content) 
        => color -> bagRegex.findAllMatchIn(content).map{m => m.group(2) -> m.group(1).toInt }.toMap
    }
    .toMap

ruleRegex: scala.util.matching.Regex = (\w+ \w+) bags contain (.*)
bagRegex: scala.util.matching.Regex = (\d+) (\w+ \w+) bag
containRules: Map[String, Map[String, Int]] = Map(
  "dull violet" -> Map(
    "posh coral" -> 5,
    "shiny red" -> 5,
    "drab gold" -> 5,
    "posh white" -> 4
  ),
  "mirrored red" -> Map("striped gold" -> 3, "mirrored gray" -> 1),
  "posh fuchsia" -> Map("wavy gold" -> 1, "vibrant plum" -> 1, "posh white" -> 3),
  "dull salmon" -> Map(
    "muted coral" -> 5,
    "muted brown" -> 2,
    "striped coral" -> 2,
    "dark white" -> 5
  ),
  "vibrant beige" -> Map(
    "pale crimson" -> 2,
    "pale olive" -> 5,
    "posh yellow" -> 4,
    "dotted orange" -> 3
  ),
  "dim green" -> Map(
    "striped lime" -> 5,
    "bright gold" -> 4,
    "vibrant violet" -> 2,
    "muted beige" -> 2
  ),
  "shiny yellow" -> Map("dim purple" -> 3),
  "faded gray" -> Map(
    "bright maroon" -> 3,
    "shiny crimson" -> 2,
    "plaid lavender" -> 2
  ),
  "wavy gray" -> Map(
    

In [6]:
def countBags(bag: String, rules: Map[String, Map[String, Int]]): Int = {
    rules.getOrElse(bag, Map()).map{case (b, c) => c * (1 + countBags(b, rules))}.sum
}

defined function countBags

In [7]:
countBags("shiny gold", containRules)

res6: Int = 35487